<a href="https://colab.research.google.com/github/yirlenarias/TFG_PD/blob/main/Datos/Observado/Temperatura/Temperatura_Observada_20230501.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Climatología de Temperatura Observada

* Para trabajar más cómodamente vamos a descargar todos los ficheros del repositorio GitHub
* De esa manera dispondremos de los datos localmente para nuestro análisis.

In [ ]:
# Directorio de trabajo local en la nube de Google Colab 
# ------------------------------------------------------------
%cd /content

# Si existe una copia previa del repositorio, la borramos:
# ----------------------------------------------------------------------------
!  rm -rf TFG_PD

# Creamos una copia del repositorio SOLO si no existe previamente
# ----------------------------------------------------------------------------
! [ ! -d TFG_PD ] && git clone https://github.com/yirlenarias/TFG_PD.git

# Entramos en el repositorio que acabamos de copiar
# --------------------------------------------------
%cd TFG_PD/Datos/Observado/Temperatura

## Importar librerias

In [ ]:
import glob
import pandas as pd
import numpy as np

## Abrir todos los archivos excel por cuenca

In [ ]:
# Criterios de coincidencia de patrones para archivos de Excel en el directorio
filenames = 'Cuenca*.xlsx'

In [ ]:
# Crea un diccionario vacío para almacenar los datos de cada archivo
dat = {}

# Recorra cada archivo de Excel que coincida con el patrón y lea sus datos en un marco de datos de pandas
for file in sorted(glob.glob(filenames)):
    dat[file] = pd.read_excel(file)

In [ ]:
data_new = []

for file, data in dat.items():
    data_new.append(data)

In [ ]:
# Definir una función para aplicar a los datos y en la columna 'ESTACION' todos los valores tengan 3 digitos
for i, df in enumerate(data_new):
    def formatear_valor(valor):
        if valor < 10:
            return '00' + str(valor)
        elif valor < 100:
            return '0' + str(valor)
        else:
            return str(valor)

    # Aplicar la función a la tabla
    data_new[i]['ESTACION'] = data_new[i]['ESTACION'].apply(formatear_valor)

## Arreglar datos

In [ ]:
#Unir el numero de 'CUENCA' con 'ESTACION' para tener el numero de cada estacion
for i, df in enumerate(data_new):
    data_new[i]['ESTACION_NEW'] = data_new[i]['CUENCA'].astype(str) + data_new[i]['ESTACION'].astype(str)

In [ ]:
#Eliminar las columnas que no son necesarias
for i, df in enumerate(data_new):
    data_new[i].pop('CUENCA')
    data_new[i].pop('ESTACION')

## Agrupar datos por estación

In [ ]:
# Agrupar las filas según los valores de la columna "ESTACION_NEW"
my_list = list(range(0, 22))
porestacion=[]
for i in my_list:
    grupos = data_new[i].groupby('ESTACION_NEW')

    # Crear una tabla separada para cada grupo segun 'ESTACION_NEW'
    for ESTACION, grupo in grupos:
        # Obtener el DataFrame correspondiente a cada grupo
        tabla = grupos.get_group(ESTACION)
        porestacion.append(tabla) #Adjuntar los valores

* Obtener el número de cada estación

In [ ]:
NUMERO = porestacion.copy()
for i, df in enumerate(NUMERO):
    NUMERO[i] = NUMERO[i].iloc[0,3]

## Guardar un archivo por cada estación

In [ ]:
# Comprimimos los archivos en un archivo ZIP
with zipfile.ZipFile('archivos_excel.zip', 'w') as zip_file:

    # Iteramos sobre las tablas y los nombres de archivos
    for tabla, nombre_archivo in zip(porestacion, NUMERO):

        # Agregamos la extensión .xlsx al nombre del archivo
        nombre_archivo += '.xlsx'

        # Guardamos la tabla en un archivo Excel con el nombre correspondiente
        tabla.to_excel(nombre_archivo, index=False)

        # Agregamos el archivo al archivo ZIP
        zip_file.write(nombre_archivo)

        # Borramos el archivo Excel
        os.remove(nombre_archivo)

## Obtener fecha inicial y final de cada estación

In [ ]:
for i, df in enumerate(porestacion):
    porestacion[i] = porestacion[i].set_index('FECHA')

In [ ]:
# Encontrar los valores de inicio y final de las fechas
my_list = list(range(0, 154))
porestacion_new=[]
for i in my_list:
    fecha_inicial = porestacion[i].index.min()
    fecha_final = porestacion[i].index.max()
    name = NUMERO[i]
    porestacion_new.append((fecha_inicial,fecha_final, name))

* Guardar el rango de fechas en un archivo .txt

In [ ]:
# Abrir el archivo en modo escritura
with open("rango_fechas.txt", "w") as archivo:
    # Escribir la lista en el archivo
    for valor in porestacion_new:
        archivo.write(str(valor) + "\n")

## Obtener los estadisticos

In [ ]:
#Eliminar las columnas que no se necesitan
for i, df in enumerate(porestacion):
    porestacion[i].pop('ESTACION_NEW')

* Separar los datos por temperatura maxima, minima y promedio

In [ ]:
TempMAX=[]
TempMIN=[]

for tabla in porestacion:
    columna1 = tabla.iloc[:, 0]
    columna2 = tabla.iloc[:, 1]
    TempMAX.append(columna1)
    TempMIN.append(columna2)

In [ ]:
type(TempMAX[0])

pandas.core.series.Series

* Crear una lista de las temperaturas Maxima, Minima y Promedio
* Los siguientes pasos se reliazan de la forma larga ya que cada estación no tiene el mismo rango de años

In [ ]:
#Crear una lista vacía llamada lista_de_listas_de_tablas
lista_de_listas_de_tablas = []

# Agregar las dos listas de tablas a la lista_de_listas_de_tablas
lista_de_listas_de_tablas.append(TempMAX)
lista_de_listas_de_tablas.append(TempMIN)

* Obtener los estadisticos mean, max y min, de cada T(prom, max, min) y cada estación

In [ ]:
# Creamos una lista vacía para almacenar los resultados
lista_resultados = []

# Iteramos a través de cada lista en la lista de listas de tablas
for lista_tablas in lista_de_listas_de_tablas:
    # Creamos una lista vacía para almacenar los resultados de esta lista de tablas
    lista_resultados_tablas = []
    # Iteramos a través de cada tabla en la lista de tablas
    for tabla in lista_tablas:
        # Agrupamos los datos por día y mes y calculamos la media, el máximo y el mínimo
        tabla_agrupada = tabla.groupby(tabla.index.strftime("%m%d")).agg(['mean', 'max', 'min'])
        # Agregamos la tabla agrupada a la lista de resultados para esta lista de tablas
        lista_resultados_tablas.append(tabla_agrupada)
    # Agregamos la lista de resultados para esta lista de tablas a la lista de resultados general
    lista_resultados.append(lista_resultados_tablas)

* Ahora separamos los datos, cada columna la convertimos en una tabla diferente
* Ejemplo de la columna mean saldran 3 tablas (mean.prom, mean.max y mean.min)

In [ ]:
# Lista de columnas a procesar
columnas = ['mean', 'max', 'min']

# Lista de listas de tablas vacía para cada columna
lista_nueva = [[] for i in range(len(columnas))]

# Bucle para iterar sobre cada columna
for i, col in enumerate(columnas):
    # Lista de listas de tablas vacía para la columna actual
    lista_col = []
    # Bucle para iterar sobre cada tabla de la lista original
    for lista_tablas in lista_resultados:
        # Lista de tablas vacía para la tabla actual
        lista_tabla_nueva = []
        # Bucle para iterar sobre cada tabla de la lista original
        for tabla in lista_tablas:
            # Selección de la columna actual y creación de una nueva tabla
            tabla_nueva = tabla[[col]]
            # Agregar la nueva tabla a la lista de tablas para la tabla actual
            lista_tabla_nueva.append(tabla_nueva)
        # Agregar la lista de tablas para la tabla actual a la lista de listas de tablas para la columna actual
        lista_col.append(lista_tabla_nueva)
    # Agregar la lista de listas de tablas para la columna actual a la lista de listas de tablas para todas las columnas
    lista_nueva[i] = lista_col

* Se selecciona solo la columna que necesitamos, para que la fecha no cause problemas

In [ ]:
# Iterar sobre cada lista en la lista principal
lista_de_listas_de_columnas = []
for lista_de_tablas in lista_nueva:
    listas_de_columnas = []
    for lista in lista_de_tablas:
        columnas = []
        for tabla in lista:
            df = tabla.iloc[:, 0]
            columnas.append(df)
        listas_de_columnas.append(columnas)
    lista_de_listas_de_columnas.append(listas_de_columnas)

* Volvemos un dataframe cada estadistico.T(prom, max, min) y luego transponemos cada dataframe

In [ ]:
# Creamos una lista vacía para almacenar los nuevos DataFrames
lista_de_dataframes = []

# Recorremos cada lista en la lista de listas de listas de tablas
for lista_de_listas_de_tablas in lista_de_listas_de_columnas:
    
    # Creamos una lista vacía para almacenar los nuevos DataFrames de esta lista
    lista_de_dataframes_de_esta_lista = []
    
    # Recorremos cada tabla dentro de cada lista
    for tabla in lista_de_listas_de_tablas:
        
        # Convertimos la tabla en un DataFrame y la transponemos
        df = pd.DataFrame(tabla).transpose()
        
        # Agregamos el nuevo DataFrame a la lista de DataFrames de esta lista
        lista_de_dataframes_de_esta_lista.append(df)
    
    # Agregamos la lista de DataFrames de esta lista a la lista general
    lista_de_dataframes.append(lista_de_dataframes_de_esta_lista)

* Renombrar las columnas

In [ ]:
# Cambiar los nombres de las columnas en cada DataFrame
for lista in lista_de_dataframes:
    for df in lista:
        df.columns = NUMERO

## Guardar estadisticos en excel

In [ ]:
# Crear el objeto ExcelWriter
writer = pd.ExcelWriter('Climatologia_Diaria_Observada_Temperatura_20230429.xlsx', engine='xlsxwriter')

# Escribir los DataFrames en diferentes hojas
lista_de_dataframes[0][0].to_excel(writer, sheet_name='mean_tmax', index=True)
lista_de_dataframes[1][0].to_excel(writer, sheet_name='max_tmax', index=True)
lista_de_dataframes[2][0].to_excel(writer, sheet_name='min_tmax', index=True)
lista_de_dataframes[0][1].to_excel(writer, sheet_name='mean_tmin', index=True)
lista_de_dataframes[1][1].to_excel(writer, sheet_name='max_tmin', index=True)
lista_de_dataframes[2][1].to_excel(writer, sheet_name='min_tmin', index=True)

# Guardar el archivo Excel
writer.save()

## Abrir lista de estaciones

In [ ]:
#Cargar la lista de estaciones que contienen el rango de fechas
estaciones = pd.read_excel(r'/Users/arias/OneDrive/Documentos/UCR/TFG/Datos/Observada/Temperatura/Lista de estaciones.xlsx')

In [ ]:
estaciones

,NUMERO,NOMBRE,LATITUD NORTE,LONGITUD OESTE,ALTITUD (msnm),INICIO,FINAL,ESTACION
0,100633,FINCA EL PATIO. PUERTO JIMENEZ,"08º 36' 04.6""","83º25'52.2""",24,2003-10-17,2022-12-31,100633
1,100641,COOPEAGROPAL. LAUREL .,"08º 28' 17.6""","82º51'38.4""",38,2007-03-11,2022-12-31,100641
2,100643,RIO CLARO,"08º 40' 26.5""","83º03'43.2""",51,2009-11-25,2022-12-31,100643
3,100649,FUNDACION NEOTROPICA. RINCON DE OSA,"08º 42' 2.9""","83º30'49.4""",80,2013-01-29,2022-11-10,100649
4,100651,COTO 49. CIUDAD NEILY,"08º 37' 52.3""","82º58'00.7""",30,2013-05-17,2022-12-31,100651
...,...,...,...,...,...,...,...,...
149,98095,ALTAMIRA,"09º 01' 45.7""","83º00'28.7""",1370,2002-01-01,2022-12-31,98095
150,98097,MONTECARLO. PZ,"09º 21' 28""","83º36'03""",1005,2008-06-18,2022-12-31,98097
151,98105,PLANTA DE VOLCAN. BUENOS AIRES,"09º 12' 15""","83º27'2.49""",416,2013-11-22,2022-12-07,98105
152,98107,CAFETALERA EL INDIO. SAN VITO. PUNTARENAS,"08º 49' 57.9""","82º57'10.5""",1027,2016-12-16,2022-11-10,98107
